In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Begin with foundations - tensors**

In [2]:
import torch
x = torch.tensor ([[1,2,3], [4,5,6]])
y = torch.tensor ([[7,8,9], [10,11,12]])
z = x + y
print (z)

tensor([[ 8, 10, 12],
        [14, 16, 18]])


**Create the tensors**

In [3]:
import numpy 

# created from preexisting arrays
w = torch.tensor ([1,2,3])                 # from a list
w = torch.tensor ([1,2,3])                 # from a tuple
w = torch.tensor (numpy.array ([1,2,3]))   # from a numpy.array

# initialized by size
w = torch.empty (100, 200)                 # uninitialized, element values are not predictable
w = torch.zeros (100, 200)                 # all elements initialized with 0.0
w = torch.ones (100, 200)                  # all elements initialized with 1.0

# initialized by size with random values
w = torch.rand (100, 200)
w = torch.randn (100, 200)
w = torch.randint (5, 10, (100, 200))

# initialized to have the same size, data type and device as another tensor
x = torch.empty_like (w)

# specify the data type at creation using dtype
w = torch.tensor ([1,2,3], dtype = torch.float32)

# use the casting method to cast to a new data type
w.int ()        # w remains a float 32 after the cast
w = w.int ()    # w changes to an int32 after the cast

# use the to() method to cast to a new type
w = w.to (torch.float64)
w = w.to (dtype = torch.float64)

# Python automatucally converts data types during operations
x = torch.tensor ([1,2,3], dtype = torch.int32)
y = torch.tensor ([1,2,3], dtype = torch.float32)
z = x + y
print (z.dtype)

torch.float32
